In [ ]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Code Toggle"></form>''')

# 목차

#### 서론

#### 사전 데이터 수집 및 가공

1.1 기사 파싱

1.2 수집한 뉴스 기사 클렌징

1.3 토큰화

#### 분석

2.1. 전체 기사 시계열 그래프

2.2. 검색 엔진으로 보는 트랜드 

2.3 LDA 토픽 모델링

2.4 LDA 모델링을 활용한 토픽 분류 및 시각화

2.5 Word2Vec을 활용한 연관어 확인 및 시각화

#### 결론

---

---

# 서론

---

현재 대한민국은 코로나 바이러스 감염증(COVID-19)으로 인한	팬데믹 상태에 빠져있습니다.  이로 인해 사람들이 살아가는 방식 또한 변화되었습니다.

코로나 바이러스 감염증 관련 뉴스 기사와 댓글을 정형화하고 이를 시계열 분석하여 생활, 문화의 변화를 확인하고, 사용자가 보기 쉽게 가시화하는 것을 목표로 합니다.

---

데이터의 저장 및 관리는 구글 드라이브를 이용합니다. 

Colab에 제공되는 패키지를 활용하여 구글드라이브에 접근할 수 있습니다.

최초 실행 시에 드라이브를 마운트 후 작업을 수행합니다.

In [ ]:
# 최초 경로 설정

root = '/content/drive/Shared drives/Corona Effect/case8/'

In [ ]:
#  구글 드라이브 마운트

import os, sys
from google.colab import drive

drive.mount('/content/drive', force_remount=True)
os.symlink(root, '/content/corona') 

root = '/content/corona/'

%cd {root}

In [ ]:
# !pip install --upgrade -q numpy
# !pip install --upgrade -q pandas
# !pip install --upgrade -q gensim
# !pip install --upgrade -q pyLDAvis

최종 디렉터리 구조는 아래와 같습니다.

In [ ]:
!apt-get install -qy tree

In [ ]:
!tree {root} -L 2 

/content/corona/
├── cleansing
├── conclusion_dupliate.ipynb
├── conclusion.ipynb
├── corona_count_modify.csv
├── Dominant_Topic.csv
├── google.xlsx
├── lda
├── lda72
├── lda72_tokenized_all
├── naver.xlsx
├── newscsv
├── pyLDAvis.html
├── tokenized
└── w2v.model

6 directories, 8 files


---

# 1. 사전 데이터 수집 및 가공

## 1.1 기사 파싱

먼저 뉴스 기사를 파싱합니다. 

뉴스 기사는 네이버 뉴스를 수집합니다. 네이버에서는 언론사별, 날짜별로 기사를 검색할 수 있고 네이버 검색은 검색 결과를 최대 4000건까지 표시합니다. 

`코로나` 키워드로 네이버 뉴스를 검색하여 주요 언론사 24곳에서 코로나 관련 기사를 수집하였습니다. 언론사 / 2일 단위로 뉴스를 검색한 뒤 뉴스를 모두 수집하여 보름 단위로 저장하였습니다.


<br/>

---

<br/>

경로 : newscsv/

확장자 : csv

수집 대상 : 네이버 뉴스

기간 : 2020년 01월 01일 ~ 2020년 08월 31일

검색 키워드 : `코로나`

대상 언론사: 네이버 뉴스 포맷을 지원하는 방송 / 신문사 24곳

데이터 : 

|     title     |      date      |          category          |   text    |                press                |   link    |
| :------------: | :------------: | :------------------------: | :-------: | :---------------------------------: | :-------: |
| 뉴스 기사 제목 | 기사 작성 일자 | 네이버에서 분류한 카테고리 | 기사 본문 | 네이버에서 사용되는 언론사 고유번호 | 기사 링크 |



[뉴스 파싱 코드](https://github.com/elppaaa/naver-news-crawler/blob/main/crawl.py)

In [ ]:
import os, sys
import pandas as pd

In [ ]:
# 보름 단위로 수집하기 위해서 배열 생성


dateset = [('0101','0115'), ('0116','0131'), ('0201','0215'), ('0216', '0229'), ('0301','0315'), ('0316','0331'), 
('0401','0415'), ('0416','0430'), ('0501','0515'), ('0516', '0531'), ('0601','0615'), ('0616','0630'),
('0701','0715'), ('0716','0731'), ('0801','0815'), ('0816','0831')]

In [ ]:
# 폴더에 있는 파일 목록을 가져오는 메서드
def get_filelist(path, extension):
  flist = os.listdir(path)
  result = []
  if path.endswith('/') == False:
    path += '/'
  for file in flist:
    if file.endswith(f'.{extension}'):
      result.append(path+file)
  return sorted(result)


In [ ]:
# 네이버 뉴스를 파싱하는 메서드 선언
# 미리 작성한 파싱 코드를 깃에서 불러와 실행
def crawl_news(l: tuple):
    os.makedirs(f'{l[0]}-{l[1]}', exist_ok=True)
    os.chidir(f'{l[0]}-{l[1]}')

    !git clone https://github.com/elppaaa/naver-news-crawler.git
    !mv naver-news-crawler/crawl.py ./
    !rm -r naver-news-crawler
    
    start = '2020' + l[0]
    end = '2020' + l[1]
    %time %run crawl.py 코로나 {start} {end} 2
    !rm crawl.py


# 언론사 / 일 단위로 수집된 뉴스 기사를 보름 단위로  합치는 메서드  
def merge_news(d: str):
    df = pd.DataFrame()
    flist = get_filelist('../newscsvraw/' + d, 'csv')
    
    for file in flist:
        tmp = pd.read_csv(file)
        df = pd.concat([df, tmp])
    df.to_csv(f'{d}.csv' ,index=False)

In [ ]:
# 뉴스 크롤러 실행
for v in dateset:
    os.chdir(root +'newscsvraw')
    crawl_news(v)

# 크롤링한 뉴스 보름 단위로 저장
for l in dateset:
    name = f'{l[0]}-{l[1]}'
    os.chdir(root + 'newscsv')
    merge_news(name)

In [ ]:
pd.read_csv(get_filelist(root + 'newscsv', 'csv')[0]).head()

,title,date,category,text,press,link
0,공사대금 문제로 시공사 대표 폭행한 전직 프로야구 선수 체포,2020-01-01 14:36:45,사회,\t\t\t(서울=연합뉴스) 권선미 기자 = 전직 프로야구 선수가 입주 예정이던 신...,1,https://m.news.naver.com/read.nhn?mode=LSD&mid...
1,말레이시아 '음식점 흡연' 708명에 첫 벌금 부과,2020-01-03 11:32:14,세계,\t\t\t1년 계도기간 끝나고 올 1월 1일부터 일제 단속(자카르타=연합뉴스) 성...,1,https://m.news.naver.com/read.nhn?mode=LSD&mid...
2,"미스트롯 전국투어 의정부 공연, 안전 문제로 취소",2020-01-05 16:46:19,생활,"\t\t\t공연기획사 ""무대 설치 도중 구조물 일부 무너져""원본보기미스트롯 전국투어...",1,https://m.news.naver.com/read.nhn?mode=LSD&mid...
3,국내서 '중국 원인불명 폐렴' 증상자 발생…36세 중국 여성(종합2보),2020-01-08 18:42:46,사회,"\t\t\t지난달 우한시 방문 후 입국…현재 격리치료 중·건강상태 양호질본, 폐렴 ...",1,https://m.news.naver.com/read.nhn?mode=LSD&mid...
4,국내서 '중국 원인불명 폐렴' 증상자 발생…36세 중국 여성(종합),2020-01-08 18:01:20,사회,"\t\t\t지난달 우한시 방문 중국인 여성…현재 건강상태 양호질본, 폐렴 유발 원인...",1,https://m.news.naver.com/read.nhn?mode=LSD&mid...


---

## 1.2 수집한 뉴스 기사 클렌징

수집된 기사는 바로 사용하기 어렵습니다.

뉴스 기사와 같은 비정형 데이터의 경우에는 수집 과정에서 불필요한 데이터들이 포함되어 있을 수 있으며 이를 사전에 제거하지 않으면 차후 분석 과정에서 노이즈가 생길 수 있습니다. 

해당 단계에서는 수집된 뉴스 기사의 본문 중에서 불필요한 텍스트를 제거하는 과정이라고 할 수 있습니다.

정규표현식과 문자열 객체에 제공되는 함수를 이용하여 불필요한 텍스트를 검색한 뒤 제거하였습니다.

또한 뉴스의 건수가 너무 많고, 불필요한 주제의 뉴스도 포함되어 있다고 판단하여 주제에 관한 내용이 포함되어 있다고 생각하는 카테고리를 선정하였습니다.

<br/>

📌 특수문자의 경우 별도의 처리를 하지 않았는데 이후 토큰화 과정에서 한글만 사용하기 때문입니다.

--- 

경로 : cleansing/

확장자 : csv

사용 카테고리 : `경제`,  `사회`,  `생활`,  `세계`

단일 파일 : cleansing_full.csv

데이터 : 

|     title     |      date      |          category          |   text    |                press                |   link    |
| :------------: | :------------: | :------------------------: | :-------: | :---------------------------------: | :-------: |
| 뉴스 기사 제목 | 기사 작성 일자 | 네이버에서 분류한 카테고리 | 전처리된 기사 본문 | 네이버에서 사용되는 언론사 고유번호 | 기사 링크 |





In [ ]:
import pandas as pd
import re, os

dateset = [('0101','0115'), ('0116','0131'), ('0201','0215'), ('0216', '0229'), ('0301','0315'), ('0316','0331'), 
('0401','0415'), ('0416','0430'), ('0501','0515'), ('0516', '0531'), ('0601','0615'), ('0616','0630'),
('0701','0715'), ('0716','0731'), ('0801','0815'), ('0816','0831')]

In [ ]:

regexlist = []

# 이메일
regexlist.append( re.compile(r'\w+@[\w\.]+', flags=re.U))

# 제보하기 같은 맨 마지막 광고
regexlist.append( re.compile(r'◀[^▶]+▶', flags=re.U))
regexlist.append(re.compile(r'▶.+$', flags=re.U))

# 뉴시스
regexlist.append(re.compile(r'공감언론 뉴시스[^\>]*', flags=re.U))
# 화살표
regexlist.append(re.compile(r'☞.+$', flags=re.U))
# Copyright
regexlist.append(re.compile(r'Copyright.+$', flags=re.U))

# # /뉴스1 © News1
regexlist.append(re.compile(r'\/\w*연합뉴스', flags=re.U))


# 대괄호 제거
regexlist.append(re.compile(r'\[[^\]]*\]', flags=re.U))
# 대괄호랑 비슷한 특수문자 대괄호 ［ ］
regexlist.append(re.compile(r'［[^］]*］', flags=re.U))


# 소괄호 제거
regexlist.append(re.compile(r'\([^\)]*\)', flags=re.U))
# < > 제거
regexlist.append(re.compile(r'<[^\>]*>', flags=re.U))
# 【 】
regexlist.append(re.compile(r'【[^】]*】', flags=re.U))


#인터넷 주
regexlist.append(re.compile(r'http[^\s\(\[=]+', flags=re.U))

# 기자, 특파원
regexlist.append(re.compile('\w{2,3}\s{0,2}(기자|특파원)', flags=re.U))



replacelist = [
  '페이스북 tuney\.kr/LeYN1',
  '트위터 @yonhap_graphics',
  '현지시각',
  '원본보기',
  '© 뉴스1'
]

#  각 기사 본문을 정규표현식을 이용하여 불필요한 문자열을 제거
def textrunner(row):
  val = row['text']
  row['text'] = regexrunner(val)
  return row

# 위의 컴파일된 re 객체를 이용하여 불필요한 텍스트 제거
def regexrunner(text):
  for r in regexlist:
    text = r.sub('', str(text))
  for r in replacelist:
    text = text.replace(r, '')
  return text

In [ ]:
#  클렌징 작업
filelist = get_filelist(root + 'newscsv', 'csv')
os.makedirs(root+'cleansing', exist_ok=True)
os.chdir(root+'cleansing')

for file in filelist:
    df = pd.read_csv(open(file, 'r'))
    file = file.split('/')[-1]
    df = df.drop_duplicates().dropna()  # NaN 중복 제거
    df= df[df.category.isin(['경제',  '사회',  '생활',  '세계'])]  # 카테고리 선정
    df = df.sort_values(by=['date'], axis=0) # 시간 순서대로 정렬
    df.apply(textrunner, axis=1).drop('index', axis=1).to_csv(file, index=False)

In [ ]:
# 하나로 합치기
f = root+'cleansing/cleansing_full.csv'
os.chdir(root + 'cleansing')


for file in get_filelist(root+'cleansing', 'csv'):
  if not os.path.exists(f):
    df = pd.read_csv(open(file, 'r'))
    df['date'] = pd.todatetime(df['date'])
    df.sort_values(by=['date'], axis=0)
    df.to_csv(f, mode='w', index=False, encoding='utf-8')
  else:
    df = pd.read_csv(open(file, 'r'))
    df['date'] = pd.todatetime(df['date'])
    df.sort_values(by=['date'], axis=0)
    df.to_csv(f, mode='a', index=False, header=False, encoding='utf-8')

In [ ]:
# 제목 / 링크가 같은 문서는 같은 뉴스로 판단하여 제거
pd.read_csv(f).drop_duplicates(['link', 'title']).sort_values(['date']).dropna().to_csv(f, index=False)

In [ ]:
#  총 기사 건수 확인
for i, rows in enumerate(pd.read_csv(f, iterator=True, chunksize=50000)):
  if len(rows) < 50000:
    print(f"총 {i*50000 + len(rows)}건의 뉴스를 사용합니다.")

총 693569건의 뉴스를 사용합니다.


In [ ]:
df = pd.read_csv(f, chunksize=1000)
next(df).head()

,title,date,category,text,press,link
0,[홍찬선의 문사철 경국부민학 3] 말을 조심하고 솔선수범하라,2020-01-01 06:36:00,경제,미혹을 넘은 현명한 선택만이 살길리어왕과 오셀로가 한국 정치꾼에게 주는 채찍#1. ...,417,https://m.news.naver.com/read.nhn?mode=LSD&mid...
1,공사대금 문제로 시공사 대표 폭행한 전직 프로야구 선수 체포,2020-01-01 14:36:45,사회,\t\t\t = 전직 프로야구 선수가 입주 예정이던 신축 빌라의 공사 대금 문제로...,1,https://m.news.naver.com/read.nhn?mode=LSD&mid...
2,[스마트 리빙] 미세먼지 심한 날엔 독감 더 잘 걸려요,2020-01-02 07:44:46,경제,\t\t\t미세먼지가 심한 날에는 독감과 같은 호흡기 감염 질환에 걸리지 않도록 더...,214,https://m.news.naver.com/read.nhn?mode=LSD&mid...
3,길가에서 70대 할머니 폭행한 30대 붙잡혀,2020-01-02 09:00:02,사회,경기 부천 한 아파트 주차장에서 70대 노인을 폭행한 30대 남성이 경찰에 붙잡혔...,56,https://m.news.naver.com/read.nhn?mode=LSD&mid...
4,"소비자기본법 40주년…공정위, 아이디어 공모전·학술대회 개최",2020-01-02 12:00:00,경제,"""기업·정부, 소비자 중심적으로 변화시킬 것"" = 정부세종청사 공정거래위원회. ...",3,https://m.news.naver.com/read.nhn?mode=LSD&mid...


---

## 1.3 토큰화



토큰은 문법적으로 더 이상 나눌 수 없는 언어 요소입니다.

텍스트 토큰화라는 것은 말뭉치로부터 토큰을 분리하는 작업을 의미합니다. 

영어의 경우에는 띄어쓰기로 대부분 토큰 구분이 가능하지만, 한국어는 띄어쓰기만으로는 구분이 어렵습니다. 한국어를 토큰화 할때는 형태소 분석기를 사용합니다. 한국어 형태소 분석기는 매우 다양하게 개발되어 있지만, 테스트를 통해 속도 / 정확도가 높은 `Mecab` 을 사용하였습니다.

<br/>

저장은 파이썬에서 기본으로 제공되는 `pickle` 라이브러리를 사용하였습니다. `pickle` 은 파이썬의 객체를 구조 그대로 저장할 수 있도록 도와줍니다. 

<br/>

해당 과정에서는 분석된 문장을 `Mecab`을 이용하여 토큰 단위로 쪼개어 명사만을 모아 2차원 배열 구조로 저장하였습니다.

<br/>

---

<br/>

경로 : tokenzied/

확장자 : pkl

구조 : 토큰화된 문서 배열로 이루어진 2차원 배열

In [ ]:
# 형태소 분석기 KoNLPy, Mecab dic 설치

!set -x \
&& pip install -q konlpy \
&& curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh | bash -x > /dev/null

In [ ]:
import pandas as pd
import re, os
import pickle
from konlpy.tag import Mecab

In [ ]:
# 특수문자를 제거하고 단어만 사용하기 위한 정규표현식
r = re.compile(r'[^ ㄱ-ㅣ가-힣|0-9|a-zA-Z]+', flags=re.U | re.M)

mecab = Mecab()

def clean_text(text):
    text = text.replace(".", " ").strip()
    text = text.replace("·", " ").strip()
    text = r.sub('', string=text)
    return text

# NNP, NNG, SL만 분리
def get_nouns(sentence):
  return [morphs[0] for morphs in mecab.pos(sentence) if morphs[1] in ['NNP', 'NNG',  'SL'] and len(morphs[0]) > 1]


def tokenize(df):
    processed_data = []
    # for sent in tqdm(df['text']):
    for sent in df['text']:
        sentence = clean_text(str(sent).replace('\n', '').strip())
        processed_data.append(get_nouns(sentence))
    return processed_data

In [ ]:
# tokenized 폴더에 저장
os.makedirs(root + 'tokenized', exist_ok=True)
os.chdir(root + 'tokenized')

# 전처리된 전체 문서 파일(cleansing_full.csv)로 토큰화 수행
for i, rows in enumerate(pd.read_csv(root+'cleansing/cleansing_full.csv', chunksize=50000, iterator=True)):
  with open('%02d.pkl' % i, 'wb') as f:
    pickle.dump(tokenize(rows), f)

In [ ]:
with open(root + 'tokenized/01.pkl', 'rb') as f:
  print(pickle.load(f)[:2])

[['미혹', '현명', '선택', '리어', '오셀로', '한국', '정치', '채찍', '로알드', '고조부', '셀베르그', '목사', '성령', '강림절', '노르웨이', '웨스트민스터', '가의', '예배당', '사람', '셀베르그', '목사', '성령', '강림', '설교', '공포', '교인', '이상', '목숨', '사람', '떼죽음', '지옥', '사람', '성경', '제단', '유리창', '유리창', '탈출', '셀베르그', '목사', '사람', '당시', '참사', '대서특필', '유럽', '신문', '목사', '판단력', '칭송', '위기', '방법', '생각', '무리', '진취', '시대', '영웅', '독립', '인간', '생각', '사람', '훗날', '노르웨이', '국회의원', '활동', '공공', '기관', '건물', '건축법', '제정', '도움', '셀베르그', '목사', '천재', '이야기', '로알드', '고조부', '고조부', '고조부', '위기관리', '능력', '현실', '능력', 'DNA', '어린이', '마음', '아이', '얘기', '세상', '기여', '리어', '오셀로', '맥베스', '리어', '사랑', '막내딸', '코델리아', '코델리아', '아버님', '자식', '의무', '사랑', '이상', '이하', '마음', '현명', '감언이설', '막내딸', '코델리아', '권력', '재산', '결과', '광야', '미치광이', '프랑스', '왕비', '막내딸', '도움', '막판', '권력', '코델리아', '감옥', '살해', '리어', '슬픔', '가슴', '장군', '오셀로', '마음씨', '데스', '데모', '니아', '행복', '결혼', '부하', '기수', '이아고', '거짓말', '아내', '살해', '사람', '마음', '사정', '농락', '괴물', '의심', '조금', '핏속', '작용', '유황광', '억측', '괴물', '의처증', '오셀로', '이아고', '농락', '이아고', '

---

<br/>

---

# 2. 분석

## 2.1 전체기사 시계열

`코로나` 키워드로 검색한 뉴스를 모두 수집하였고, 수집된 양은 기간별로 상이합니다.

코로나에 대한 사건이 많을 수록, 주목이 많이 될 수록 기사의 수가 많아질 것이라고 판단하였습니다.

뉴스의 기사 수를 기간 별로 비교하여 관심도를 확인하였습니다.

주말에는 작성되는 기사가 상대적으로 적어 요일별로 차이가 크기 때문에 주 단위로 집계하였습니다.

국가 통계 포털에서 제공하는 일 단위 확진자 데이터도 활용하여 기사의 건수와 함께 비교하였습니다.


In [ ]:
from bokeh.plotting import output_notebook, figure, show
from bokeh.palettes import viridis, Spectral, Paired
from bokeh.models import ColumnDataSource, DatetimeTickFormatter, HoverTool, Legend, Range1d, LinearAxis
import datetime
import pandas as pd

output_notebook()

In [ ]:
# 국가 통계 포털의 일 확진자 데이터
corona_data = pd.read_csv(root + 'corona_count_modify.csv')
corona_data['date'] = pd.to_datetime(corona_data['date'])

먼저, 국가 통계 포털에서 제공되는 일 확진자 데이터입니다. 

국가 통계 포털에서 제공되는 데이터는 그대로 사용하기에는 index, column이 적절하지 못하여 일부 header 부분만 일부 수정하였습니다. 

<br/>
<br/>

일 확진자 데이터: 

In [ ]:
corona_data.head()

,date,신규 확진,누적 확진,격리 증감,누적 격리,신규 격리해제,누적 격리 해제,신규 사망,누적 사망
0,2020-02-04,18,18,-,-,0,0,0,0
1,2020-02-05,1,19,-,-,1,1,0,0
2,2020-02-06,4,23,-,-,0,1,0,0
3,2020-02-07,1,24,-,-,1,2,0,0
4,2020-02-08,0,24,-,-,0,2,0,0


<br/>
<br/>
1주일 단위로 그룹화:

In [ ]:
corona_data = corona_data.groupby(pd.Grouper(key='date', freq='1w'))
corona_data = corona_data.sum()['신규 확진']
pd.DataFrame(corona_data).head()

,신규 확진
date,
2020-02-09,27
2020-02-16,2
2020-02-23,573
2020-03-01,3134
2020-03-08,3398


다음은 수집한 뉴스 건수입니다.

중복된 뉴스와 같은 경우를 줄이기 위해서 클렌징하여 필터링된 뉴스를 사용합니다.

위와 마찬가지로 1주일 단위로 뉴스의 그룹을 묶어 뉴스의 개수를 카운팅하였습니다.

In [ ]:
# 클렌징된 뉴스를 사용

df = pd.read_csv(root+'cleansing/cleansing_full.csv')
df['date'] = pd.to_datetime(df['date'])
df = df[ df['date'] < '2020-09-01' ]

In [ ]:
# 주 단위로 그룹을 묶어 개수를 셈

wcount = df.groupby(pd.Grouper(key='date', freq='1w')).count()
wcount = wcount[wcount.index < '2020-09-01']

In [ ]:
# 재정렬
wcount = wcount.reset_index()

In [ ]:
wcount.head()

,date,title,category,text,press,link
0,2020-01-05,14,14,14,14,14
1,2020-01-12,264,264,264,264,264
2,2020-01-19,406,406,405,406,406
3,2020-01-26,5307,5307,5299,5307,5307
4,2020-02-02,18110,18110,18095,18110,18110


시각화는 `Bokeh`를 활용하였습니다. Bokeh는 결과물이 JS로 렌더링되기 때문에 동적인 그래프를 만들 수 있습니다.


In [ ]:
# 주요 이벤트를 좌표
event = (pd.to_datetime(['2020-01-20', '2020-02-17', '2020-02-19', '2020-08-15']),
         [1617,18621, 21061, 15134])

In [ ]:
# plot 생성
plot = figure(title="주 단위 뉴스 기사 건수 집계", width=1400, height=400, x_axis_type='datetime', sizing_mode='stretch_width')

# 드래그 방지
plot.toolbar.active_drag = None

# 마우스를 올렸을 때 표시되는 HoverTool 사용
plot.add_tools(HoverTool(
    tooltips = [
      ("date", "$x{%F}"),
      ("count", "$y{D}")
    ],
    formatters={
        '$x': 'datetime'
    })
)

In [ ]:
# line Plot 그리기
p1 = plot.line(x=wcount['date'], y=wcount['title'], line_width=2)

# 코로나 관련 사건 표시
p2 = plot.circle(x=event[0][0], y=event[1][0], size=8, fill_alpha=0.7, color=Spectral[4][0])
p3 = plot.circle(x=event[0][1], y=event[1][1], size=8, fill_alpha=0.7, color=Spectral[4][1])
p4 = plot.circle(x=event[0][2], y=event[1][2], size=8, fill_alpha=0.7, color=Spectral[4][2])
p5 = plot.circle(x=event[0][3], y=event[1][3], size=8, fill_alpha=0.7, color=Spectral[4][3])
p6 = plot.line(x = corona_data.index, y = corona_data, y_range_name = 'corona', line_alpha=0.2, line_width=2, color="purple")

In [ ]:
# 범례 표시
plot.add_layout(
    Legend(items=[
                  ('기사 건수',[p1]),
                  ('한국에서의 첫 확진자 발생\n (1월20일)',[p2]),
                  ('신천지에서의 슈퍼확진자 발생\n (2월17일)',[p3]),
                  ('대구 및 경북지역의 집단감염\n (2월19일)', [p4]),
                  ('광화문에서의 집단 집회\n (8월15일)', [p5]),
                  ('코로나 신규 확진자', [p6])
], location="center"), "right")

In [ ]:
# 확진자 수의 y축 추가
plot.extra_y_ranges = {"corona": Range1d(start=0, end=max(corona_data.values))}
plot.add_layout(LinearAxis(y_range_name="corona", axis_label="확진자 수, 기사 수"), 'left')

In [ ]:
show(plot)

국가 통계 포털에서 2020/02/04 이후의 확진자 정보를 확인할 수 있었습니다.

해당 값은 모두 주 단위의 값입니다

2-3월 신천지 대구교회 관련 집단 감염 사건 당시 뉴스 기사 수도 급증하는 것을 볼 수 있습니다. 

이후에는 감소세를 보이다가 8월 수도권 중심의 2차 대유행 시기에 확진자가 급증하면서 뉴스의 건수도 급증했습니다. 

뉴스 기사 건수와 신규 확진자 수를 보면 거의 비례하게 증가하는 것을 볼 수 있습니다.




---

## 2.2 검색 엔진 트랜드

네이버와 구글에서는 운영하는 검색 엔진을 바탕으로 트랜드가 어떻게 변화하는지를 확인할 수 있는 서비스를 제공하고 있습니다.

네이버는 네이버 데이터 랩에서, 구글은 구글 트랜드에서 동일한 키워드를 이용하여 검색량의 변화를 확인하였습니다.

비교를 위하여 코로나 유행 이전의 2019년 11월, 12월의 자료를 포함하여 차트를 확인하였습니다.


In [ ]:
from bokeh.plotting import output_notebook, figure, show
from bokeh.palettes import viridis, Spectral, Paired
from bokeh.models import ColumnDataSource, DatetimeTickFormatter, HoverTool, Legend, Segment, Range1d, LinearAxis
import datetime
import pandas as pd

output_notebook()

네이버는 네이버 데이터 랩의 정보를 사용했습니다. 네이버 데이터 랩에서는 각 키워드별 트랜드를 시간 순서로 알려줍니다. 해당 데이터는 주 단위 데이터입니다. 검색어별로 검색량의 절대적인 차이가 있기 때문에 각 키워드의 검색량은 상대적 수치입니다.

In [ ]:
# 네이버 데이터랩 데이터
naver = pd.DataFrame(pd.read_excel(root + 'naver.xlsx'))
naver = naver.set_index(['date'])
naver.head()

,abroad,domestic,shopping,ott,health
date,,,,,
2019-10-07,36.31309,49.02165,39.97107,29.13172,36.78625
2019-10-14,39.51153,52.79557,42.60166,27.71416,38.08508
2019-10-21,42.69417,55.52359,42.31487,30.19902,39.23932
2019-10-28,42.30951,56.81352,43.33785,29.84043,36.86712
2019-11-04,42.88913,58.16882,45.41473,31.10288,36.68578


구글은 구글 트랜드의 데이터를 사용했습니다. 구글 트랜드에서는 각 검색 키워드의 시간별, 지역별 등 다양한 기준으로 데이터를 받아올 수 있습니다. 본 과정에서는 시간별 변동만 확인했습니다. 검색어별로 검색량의 절대적인 차이가 있기 때문에 각 키워드의 검색량은 상대적 수치입니다.

In [ ]:
# 구글 트랜드 데이터 
google = pd.DataFrame(pd.read_excel(root + 'google.xlsx'))
google = google.set_index(['date'])
google.head()

,ott,domestic,abroad,shopping,health
date,,,,,
2019-10-06,33,43,71,51,16
2019-10-13,33,26,66,51,27
2019-10-20,35,42,52,53,43
2019-10-27,32,87,67,53,11
2019-11-03,36,43,75,54,22


<br/>
<br/>

이전과 동일한 국가 통계 포털에서 제공하는 일 확진자 현황입니다.

In [ ]:
# 국가 통계 포털의 일 확진자 데이터
corona_data = pd.read_csv(root + 'corona_count_modify.csv')
corona_data['date'] = pd.to_datetime(corona_data['date'])

In [ ]:
corona_data.head()

,date,신규 확진,누적 확진,격리 증감,누적 격리,신규 격리해제,누적 격리 해제,신규 사망,누적 사망
0,2020-02-04,18,18,-,-,0,0,0,0
1,2020-02-05,1,19,-,-,1,1,0,0
2,2020-02-06,4,23,-,-,0,1,0,0
3,2020-02-07,1,24,-,-,1,2,0,0
4,2020-02-08,0,24,-,-,0,2,0,0


<br/>
이번에도 주 단위로 묶어 사용합니다.

In [ ]:
# Grouper로 주 단위로 묶어 합계를 사용
corona_data = corona_data.groupby(pd.Grouper(key='date', freq='1w'))
corona_data = corona_data.sum()['신규 확진']
pd.DataFrame(corona_data).head()

,신규 확진
date,
2020-02-09,27
2020-02-16,2
2020-02-23,573
2020-03-01,3134
2020-03-08,3398


동적인 그래프를 그리기 위해 `Bokeh`를 사용했습니다.

In [ ]:
# plot 생성, 드래그 방지
plot = figure(title="주 단위 검색량 추이(네이버)", width=1400, height=400, x_axis_type='datetime', sizing_mode='stretch_width', y_range = Range1d(start=0, end=100))
plot.toolbar.active_drag = None

# HoverTool 추가
plot.add_tools(HoverTool(
    tooltips = [
      ("날짜", "$x{%F}"),
      ("해외여행", "@abroad"),
      ("국내여행", "@domestic"),
      ("온라인 쇼핑", "@shopping"),
      ("OTT서비스", "@ott"),
      ("홈짐", "@health"),
    ],
    formatters={
        '$x': 'datetime'
    },
  )
)

In [ ]:
# 사용할 색상 가져오기
color = Paired[len(naver.columns)]

# 데이터셋 설정
source = ColumnDataSource(dict(
    x = naver.index,
    abroad = naver['abroad'],
    domestic = naver['domestic'],
    shopping = naver['shopping'],
    ott = naver['ott'],
    health = naver['health'],
))

In [ ]:
# line plot 그리기
p1 = plot.line(x = 'x', y = 'abroad', line_width=2, color = color[0], source=source)
p2 = plot.line(x = 'x', y = 'domestic', line_width=2, color = color[1], source=source)
p3 = plot.line(x = 'x', y = 'shopping', line_width=2, color = color[2], source=source)
p4 = plot.line(x = 'x', y = 'ott', line_width=2, color = color[3], source=source)
p5 = plot.line(x = 'x', y = 'health', line_width=2, color = color[4], source=source)

p6 = plot.line(x = corona_data.index, y = corona_data, y_range_name = 'corona', line_alpha=0.2, line_width=2, color="purple")

In [ ]:
# 범례 그리기
legend = Legend(items=[
              ('해외여행',[p1]),
              ('국내여행', [p2]),
              ('온라인 쇼핑', [p3]),
              ('OTT서비스', [p4]),
              ('홈짐', [p5]),
              ('코로나 신규 확진자', [p6])
], location="top_left")
legend.click_policy = 'hide'

# 오른쪽에 범례 추가
plot.add_layout(legend, 'right')

# 신규 확진자 수치 y축 설정 설정
plot.extra_y_ranges = {"corona": Range1d(start=0, end=max(corona_data.values))}
# plot.add_layout(LinearAxis(y_range_name="corona", axis_label="확진자 수"), 'left')

In [ ]:
show(plot)

네이버의 검색량 추이입니다. 여행 관련 검색어(해외/국내)는 코로나 이전 계속해서 증가세를 보이다가, 코로나에 대한 이야기가 계속 언급되던 시점인 1월 말쯤부터 급감하는 모습을 보여주었습니다. 

4월 말 경 신규 확진자가 10명대로 많이 줄어든 모습을 보여줬습니다. 4월 말인 4월 30일부터 부처님 오신날을 시작으로 긴 연휴가 있었습니다. 이에 대한 기대감으로 그 전 기간 동안 국내여행에 대한 검색어는 증가하고, 해외 여행에 대한 검색어 또한 국내 검색어 만큼은 아니지만 증가하는 것을 볼 수 있습니다.

크게 보이는 사건은 없었으나 6월 28일 전 세계 확진자 수가 천만명, 사망자 수가 오십만명을 넘었고, 이에 따라 해외여행도 감소세를 탄 것이 아닐까 추측하였습니다.


In [ ]:
# plot 생성, 드래그 방지
plot = figure(title="주 단위 검색량 추이(구글)", width=1400, height=400, x_axis_type='datetime', sizing_mode='stretch_width', y_range = Range1d(start=0, end=100))
plot.toolbar.active_drag = None

# HoverTool 추가
plot.add_tools(HoverTool(
    tooltips = [
      ("날짜", "$x{%F}"),
      ("국내여행", "@domestic"),
      ("해외여행", "@abroad"),
      ("온라인 쇼핑", "@shopping"),
      ("OTT서비스", "@ott"),
      ("홈짐", "@health"),
    ],
    formatters={
        '$x': 'datetime'
    },
    )
)

In [ ]:
# 사용할 색상 가져오기
color = Paired[len(google.columns)]

# 데이터셋 설정
source = ColumnDataSource(dict(
    x = google.index,
    abroad = google['abroad'],
    domestic = google['domestic'],
    shopping = google['shopping'],
    ott = google['ott'],
    health = google['health'],
))

In [ ]:
# line plot 그리기
p1 = plot.line(x = 'x', y = 'abroad', line_width=2, color = color[0], source=source)
p2 = plot.line(x = 'x', y = 'domestic', line_width=2, color = color[1], source=source)
p3 = plot.line(x = 'x', y = 'shopping', line_width=2, color = color[2], source=source)
p4 = plot.line(x = 'x', y = 'ott', line_width=2, color = color[3], source=source)
p5 = plot.line(x = 'x', y = 'health', line_width=2, color = color[4], source=source)

p6 = plot.line(x = corona_data.index, y = corona_data, y_range_name = 'corona', line_alpha = 0.5, line_width=1.5, color="purple")

In [ ]:
# 범례 그리기
legend = Legend(items=[
              ('해외여행',[p1]),
              ('국내여행', [p2]),
              ('온라인 쇼핑', [p3]),
              ('OTT서비스', [p4]),
              ('홈짐', [p5]),
              ('코로나 신규 확진자', [p6])
], location="top_left")
legend.click_policy = 'hide'

# 오른쪽에 범례 추가
plot.add_layout(legend, "right")

# 신규 확진자 수치 y축 설정 설정
plot.extra_y_ranges = {"corona": Range1d(start=0, end=max(corona_data.values))}

In [ ]:
show(plot)

구글 트랜드의 데이터입니다. 네이버 검색보다 가파른 모습을 보여주었는데, 이유는 찾지 못했습니다.

전체적으로 트랜드를 확인해보면, 네이버의 그것과 마찬가지로, 국내 / 해외 여행은 관심은 줄어들고, OTT 서비스, 온라인 쇼핑에 대한 관심은 꾸준히 증가한 모습을 볼 수 있습니다.



---

## 2.3 LDA

뉴스 기사를 어떤 토픽으로 나누어야 할지, 어느정도의 세분류가 필요할지 가늠을 하지 못하고, 라벨링이 된 정형 데이터가 아니므로 비지도 학습인 LDA 토픽 모델링을 수행하였습니다.

잠재 디리클레 할당(Latent Dirichlet Allocation, 이하 LDA)는 토픽 모델링에서 가장 대표적으로 사용되는 알고리즘입니다. LDA를 이용하여 문서의 집합에 어떠한 토픽들이 존재하는지 알아낼 수 있습니다. LDA에서 문서들은 토픽들의 혼합으로 구성되어 있고, 토픽들은 확률 분포에 기반하여 단어들을 생성한다고 가정합니다. LDA를 활용하여 각 문서(뉴스 기사)들이 어떠한 토픽을 가지고 있는지 확인할 수 있습니다.

토픽 모델링을 진행한 후, 결과물의 정확도를 평가해야 합니다. 비지도 학습인 LDA 모델링은 주로 Coherence와 Perplexity 두가지를 측정합니다. 이 중 Coherence를 이용하여 측정합니다.

Topic Coherence는 결과로 나온 주제들의 대해 각각의 주제에서 상위 N개의 단어를 뽑습니다. 모델링이 잘 되어있을 수록 의미론적으로 유사한 단어가 많이 모여 잇을 테니 이러한 유사도를 측정합니다. 

이후 적절한 개수의 토픽으로 LDA 모델을 채택하고 이를 시각화합니다.

LDA 모델 생성, Coherence 측정은 토픽 모델링으로 유명한 `gensim` 패키지를 활용하였고, 시각화는 `gensim` LDA 모델을 지원하는 `pyLDAvis` 패키지를 사용합니다.

In [ ]:
!pip install -q pyLDAvis

import os
import numpy as np
import gensim
import pickle
from gensim.models import TfidfModel
import pyLDAvis
import pyLDAvis.gensim
from gensim.corpora import Dictionary
from gensim.models.ldamodel import LdaModel
from gensim.models.coherencemodel import CoherenceModel
from gensim.models.ldamulticore import LdaMulticore

from bokeh.plotting import output_notebook, figure, show
from bokeh.palettes import Spectral 
# from bokeh.models import ColumnDataSource, DatetimeTickFormatter, HoverTool, Legend, Segment

In [ ]:
# 파일 가져올때 사용
def get_filelist(path, extension):
  flist = os.listdir(path)
  result = []
  if path.endswith('/') == False:
    path += '/'
  
  for file in flist:
    if file.endswith(f'.{extension}'):
      result.append(path+file)
  return sorted(result)

# 쪼개진 pickle 파일을 generator로 사용
class TokenCorpus():
  def __init__(self, flist):
    self.flist = flist

  def __iter__(self):
    for file in self.flist:
      with open(file, 'rb') as f:
        for el in pickle.load(f):
          yield el

def fake_tokenizer(e):
  return e

LDA 에서는 입력 데이터로 BoW, DTM 행렬, TF-IDF 행렬을 사용합니다. 이 중에서 BoW를 사용하여 분석을 진행하였습니다.

가장 먼저, 명사 단어들로 이루어진 Dictionary 를 생성합니다. 

이후, `Dictionary` 객체와 `doc2bow()` 메서드를 활용하여 BoW 데이터를 생성합니다.



In [ ]:
# 토큰 파일 목록
flist = get_filelist(root + 'tokenized', 'pkl')

# 제너레이터로 사용할 클래스
corpus = TokenCorpus(flist)

# dictionary, bag-of-words 생성
dic = Dictionary(corpus)
bow = [dic.doc2bow(doc) for doc in corpus]

In [ ]:
# dictionary, bow 일부 출력
print(f'dic: {dic}')
print(f'bow: {bow[:3]}')

dic: Dictionary(187974 unique tokens: ['DNA', 'GDP', '가르침', '가슴', '가의']...)
bow: [[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 2), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 4), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 2), (32, 1), (33, 1), (34, 3), (35, 5), (36, 3), (37, 2), (38, 1), (39, 1), (40, 1), (41, 2), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 2), (50, 2), (51, 2), (52, 1), (53, 1), (54, 2), (55, 2), (56, 1), (57, 2), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 2), (64, 4), (65, 1), (66, 1), (67, 1), (68, 1), (69, 2), (70, 1), (71, 6), (72, 2), (73, 3), (74, 1), (75, 3), (76, 1), (77, 3), (78, 2), (79, 7), (80, 1), (81, 1), (82, 6), (83, 2), (84, 1), (85, 1), (86, 1), (87, 1), (88, 1), (89, 1), (90, 6), (91, 1), (92, 1), (93, 1), (94, 1), (95, 1), (96, 2), (97, 1), (98, 1), (99, 1), (100, 1), (101, 1), (102, 1

`gensim` 에서 제공하는 `LdaMulticore` 클래스를 활용하면 멀티코어로 학습해 조금더 빠른 속도로 작업을 수행할 수 있습니다.

조금 더 정확한 분류를 위해 적절한 토픽의 개수로 분류 되어야 합니다. 일관성(Coherence)를 측정하여 적절한 개수로 분류될 수 있도록 합니다.

Coherence 측정을 위해 모델을 먼저 생성합니다. 

In [ ]:
%%time 
for num_topic in range(2, 81):
  # LDA 모델 생성, num_topic만 계속 변경
  model = LdaMulticore(bow, id2word=dic, chunksize=50000, num_topics=num_topic, passes=5, workers=5)
  os.mkdir(f'{root}lda/lda{num_topic}')
  print(num_topic,':', 'mkdir_ok')
  # 생성한 모델 저장
  model.save(f'{root}lda/lda{num_topic}/lda_{num_topic}.model')
  print(num_topic,':', 'model_save_ok')

2 : mkdir_ok
2 : model_save_ok
3 : mkdir_ok
3 : model_save_ok
4 : mkdir_ok
4 : model_save_ok
5 : mkdir_ok
5 : model_save_ok
6 : mkdir_ok
6 : model_save_ok
7 : mkdir_ok
7 : model_save_ok
8 : mkdir_ok
8 : model_save_ok
9 : mkdir_ok
9 : model_save_ok
10 : mkdir_ok
10 : model_save_ok
11 : mkdir_ok
11 : model_save_ok
12 : mkdir_ok
12 : model_save_ok
13 : mkdir_ok
13 : model_save_ok
14 : mkdir_ok
14 : model_save_ok
15 : mkdir_ok
15 : model_save_ok
16 : mkdir_ok
16 : model_save_ok
17 : mkdir_ok
17 : model_save_ok
18 : mkdir_ok
18 : model_save_ok
19 : mkdir_ok
19 : model_save_ok
20 : mkdir_ok
20 : model_save_ok
21 : mkdir_ok
21 : model_save_ok
22 : mkdir_ok
22 : model_save_ok
23 : mkdir_ok
23 : model_save_ok
24 : mkdir_ok
24 : model_save_ok
25 : mkdir_ok
25 : model_save_ok
26 : mkdir_ok
26 : model_save_ok
27 : mkdir_ok
27 : model_save_ok
28 : mkdir_ok
28 : model_save_ok
29 : mkdir_ok
29 : model_save_ok
30 : mkdir_ok
30 : model_save_ok
31 : mkdir_ok
31 : model_save_ok
32 : mkdir_ok
32 : model_s

2 - 80개의 모델을 모두 생성한 이후, 생성된 모델의 Coherence를 측정합니다. 

측정 또한 `gensim` 에서 제공하는 `CoherenceModel`을 사용했습니다. 

In [ ]:
coherences = []

In [ ]:
%%time 
for num_topic in range(2, 81):
  # LDA 모델 불러오기
  model_ = LdaModel.load (f'{root}lda/lda{num_topic}/lda_{num_topic}.model')
  # Coherence Model을 bow을 이용하여 측정
  cm = CoherenceModel(model=model_, corpus=bow, coherence='c_v', texts = corpus )
  coherence = cm.get_coherence()
  print(num_topic,": Coherence :",coherence)
  coherences.append(coherence)

2 : Coherence : 0.48772722392274304
3 : Coherence : 0.5024692370031295
4 : Coherence : 0.5184390771841996
5 : Coherence : 0.49047271096697126
6 : Coherence : 0.5112172922743616
7 : Coherence : 0.5213304156680164
8 : Coherence : 0.517480594886506
9 : Coherence : 0.5189379027111818
10 : Coherence : 0.5180527972651984
11 : Coherence : 0.5625508468956504
12 : Coherence : 0.5355458991677943
13 : Coherence : 0.5734285472119004
14 : Coherence : 0.543958713899022
15 : Coherence : 0.5582416796221631
16 : Coherence : 0.5607023584749189
17 : Coherence : 0.5678667324535196
18 : Coherence : 0.5815529095676452
19 : Coherence : 0.585782700030285
20 : Coherence : 0.5757239224152753
21 : Coherence : 0.5848502237690997
22 : Coherence : 0.5563004303482265
23 : Coherence : 0.5929176505540723
24 : Coherence : 0.5700702733292392
25 : Coherence : 0.6151559248423709
26 : Coherence : 0.5986191886580192
27 : Coherence : 0.5790337774768051
28 : Coherence : 0.5798824318626895
29 : Coherence : 0.597321401794611
30

결과 중 응집도가 가장 높은 모델을 채택합니다. Coherence 값은 0 - 1 사이이며, Coherence 값이 낮아도 좋지 않지만, 값이 너무 높은것도 정상적이지 않을 수 있습니다.

아래 그래프를 확인해보면, 토픽의 개수가 72(Coherence: 0.62)일때 가장 높은 것을 확인할 수 있습니다.

<br/>

📌 이후 토픽별로 분류하여 분류된 기사들을 확인하였고, 72개의 토픽으로 분류하였을 때가 가장 좋다고 판단했습니다.

In [ ]:
output_notebook()

# Hover 툴팁
tooltips = [
            ("x, y", "$x{D}, $y")
]

# plot 생성
plot = figure(title="Coherence 측정", width=1400, height=400, sizing_mode='stretch_width',  tooltips=tooltips)
plot.toolbar.active_drag = None

# line plot 그리기
plot.line(x = range(2, 81), y = coherences, line_width=2)
highest = sorted(coherences, reverse=True)[0]

# 가장 큰 숫자 표시
plot.circle(x = coherences.index(highest) + 2, y = highest, size = 8, color = "#fee08b")

show(plot)

위의 Coherence를 기준으로 가장 적절하다고 판단되는 72개의 토픽을 가진 모델을 사용했습니다.

아래는 `pyLDAvis` 패키지를 활용하여 LDA 모델을 시각화 한 결과입니다.

토픽의 번호는 분류를 수행할 때 임의로 부여되는 번호이며, 이것이 의미를 지니지는 않습니다. 토픽에 번호는 모델 생성 시마다 달라질 수 있습니다. 

In [ ]:
# 토큰화된 파일 목록 불러옴
flist = get_filelist(root + 'tokenized', 'pkl')

# 제너레이터 생성
corpus = TokenCorpus(flist)

# dictionary, bow 객체 생성
dic = Dictionary(corpus)
bow = [dic.doc2bow(doc) for doc in corpus]

# 토픽 개수 72개의 lda 모델 불러옴
model_72 = gensim.models.LdaModel.load (f'{root}lda/lda72/lda_72.model')

In [ ]:
# pyLDAvis 패키지를 이용해 LDA 시각화
pyLDAvis.enable_notebook()

vis = pyLDAvis.gensim.prepare(model_72, bow, dic, sort_topics=False)
pyLDAvis.save_html(vis, 'pyLDAvis.html')
pyLDAvis.display(vis)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
return None

`pyLDAvis` 패키지를 활용하여 LDA 모델링 결과를 시각화 하였습니다. 위에서 측정했던 모델중 가장 적절한 토픽 개수라고 판단한 72개의 토픽으로 생성한 모델을 채택하였고, 이를 시각화 하였습니다.

왼쪽은 topic vector를 `PCA` 를 통해 2차원으로 축소시킨 결과이고, 오른쪽에는 각 토픽에서 사용하는 키워드입니다. 


오른쪽 위에 있는 슬라이드를 통해 상관 관계를 조절할 수 있습니다. 슬라이드가 1에 가까울수록 해당 토픽에 자주 출현한 단어이고, 0에 가까울수록 해당 토픽에만 존재하는 단어를 의미합니다.






각 토픽들의 대표적인 단어들도 대체로 연관성있게 보여주어 우려보다는 괜찮은 결과를 확인했습니다.

LDA 토픽 모델링에서 유사한 토픽들은 좌표 또한 인접하게 보여주는데, 
선별하려고 하였던 토픽들이 대체로 뭉쳐있는 것을 확인할 수 있었습니다.

---

## 2.4 LDA 모델링을 활용한 토픽 분류 및 시각화

위 2.3에서는 토픽의 개수를 2 - 80 개로 테스트하여, LdaModel 토픽의 개수를 정했습니다.

앞서 Coherence 스코어가 토픽 개수가 72일때, 가장 높아 채택하였지만, 조금 더 정확하게 확인하기 위해 각 문서에서 가장 비중이 높은 토픽을 기준으로 삼아 72가지로 문서를 분류 후 각 토픽을 확인했습니다.

토픽의 번호는 0번부터 71번까지 총 72개입니다.



In [ ]:
from gensim.models.ldamodel import LdaModel
from gensim.corpora import Dictionary
import pandas as pd
import numpy as np
import pickle

문서가 가지고 있는 토픽 중, 가장 비중이 높은 토픽을 선정합니다.

분류가 잘 되었는지 확인하기 위해 각 문서의 지배적인 토픽을 확인하고 해당 토픽으로 문서들을 분류했습니다.

In [ ]:
def format_topics_sentences(ldamodel, corpus):
    df = pd.DataFrame()
    # ldamodel[bow]로 모든 문서들의 토픽 비중 목록을 읽음
    for i, row in enumerate(ldamodel[corpus]):
        # 토픽 분포중 가장 높은 토픽을 사용
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        if row is not None:
          topic_num, prop_topic = row[0]
          df = df.append(pd.Series([int(topic_num), round(prop_topic,4)]), ignore_index=True)

    df.columns = ['Dominant_Topic', 'Perc_Contribution']
    return(df)

In [ ]:
# 파일 목록 가져오기
def get_filelist(path, extension):
  flist = os.listdir(path)
  result = [ ]
  if path.endswith('/') == False:
    path += '/'
  
  for file in flist:
    if file.endswith(f'.{extension}'):
      result.append(path+file)
  return sorted(result)

# 토큰 목록 제너레이터
class TokenCorpus():
  def __init__(self, flist):
    self.flist = flist

  def __iter__(self):
    for file in self.flist:
      with open(file, 'rb') as f:
        for el in pickle.load(f):
          yield el

def fake_tokenizer(e):
  return e

In [ ]:
# 토큰화된 파일 목록으로 dic, bow 생성
flist = get_filelist(root + 'tokenized', 'pkl')

corpus = TokenCorpus(flist)

dic = Dictionary(corpus)
bow = [dic.doc2bow(doc) for doc in corpus]

In [ ]:
# lda model 읽기
model = LdaModel.load(root + 'lda/lda72/lda_72.model')

In [ ]:
# 지배적인 토픽 불러오기
df = format_topics_sentences(ldamodel=model, corpus=bow)

# 재정렬, column 설정
df = df.reset_index()
df.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib']

In [ ]:
# dataframe 저장
df.to_csv(root + 'Dominant_Topic.csv', index=False)

In [ ]:
df = pd.read_csv(root + 'Dominant_Topic.csv')

In [ ]:
# { 토픽: [문서번호] } 로 딕셔너리 생성
topic_dict = dict()

for row in df.iterrows():
  v = int(row[1]['Dominant_Topic'])
  i = int(row[0])
  if  v in topic_dict:
    topic_dict[v].append(i)
  else:
    topic_dict[v] = [i]

In [ ]:
# lda72 폴더에 분류된 파일 저장
os.makedirs(root + 'lda72', exist_ok=True)
os.chdir(root + 'lda72')

# 생성한 딕셔너리를 기준으로 분류 수행
# 뉴스 기사를 토픽별로 csv 파일에 저장
for key in topic_dict.keys():
  tmp = pd.DataFrame()
  for i, rows in enumerate(pd.read_csv(root + 'cleansing/cleansing_full.csv', chunksize=50000, iterator=True)):
    tmp = pd.concat([tmp, rows[rows.index.isin(topic_dict[key])]  ])
  tmp.to_csv("%02d.csv" % key) 

In [ ]:
sorted(os.listdir(root + 'lda72'))[:10]

['00.csv',
 '01.csv',
 '02.csv',
 '03.csv',
 '04.csv',
 '05.csv',
 '06.csv',
 '07.csv',
 '08.csv',
 '09.csv']

72개의 토픽으로 토픽 모델링을 수행했을 때의 각 모델의 대표적인 키워드입니다.

In [ ]:
# 각 토픽별 키워드
# 딕셔너리와 model으로 불러오기
corpus = TokenCorpus(flist)
dic = Dictionary(corpus)

model = LdaModel.load(root + 'lda/lda72/lda_72.model')

# 각 토픽 대표 단어를 출력
for i in range(72):
  print(f"{i}:\t{[dic[t[0]] for t in model.get_topic_terms(i, topn=20)]}")

0:	['일본', '도쿄', '코로나', '긴급', '사태', '정부', '감염', '신문', '크루즈', '감염자', 'NHK', '선언', '보도', '확인', '신종', '프린세스', '바이러스', '감염증', '다이아몬드', '이날']
1:	['환자', '확진', '치료', '코로나', '경남', '완치', '퇴원', '입원', '판정', '병원', '격리', '추가', '사망자', '남성', '이날', '국내', '발생', '여성', '바이러스', '신종']
2:	['코로나', '환자', '질환', '교수', '병원', '감염', '건강', '사망', '치료', '이상', '경우', '증상', '결과', '위험', '어린이', '중증', '의학', '대남', '필요', '의료']
3:	['확진', '판정', '코로나', '검사', '방문', '오후', '조사', '거주', '접촉자', '남성', '동선', '여성', '경기도', '이날', '보건소', '격리', '오전', '양성', '감염증', '신종']
4:	['홍콩', '시장', '중국', '베이징', '코로나', '무증상', '감염자', '교도소', '확진', '모닝', '당국', '발생', '보안법', '사우스', '본토', '차이나', '포스트', '국가', '위생', '건강']
5:	['직원', '센터', '코로나', '근무', '서울', '폐쇄', '물류', '확진', '재택근무', '바이러스', '감염증', '신종', '선별', '쿠팡', '진료소', '업무', '방역', '오전', '건물', '출근']
6:	['기업', '사업', '산업', '투자', '그룹', '회장', '기술', '경영', 'SK', '혁신', '대표', '글로벌', '회사', '부회장', '분야', '삼성', '개발', '전략', '성장', '미래']
7:	['금융', '시장', '투자', '부동산', '금리', '자산', '주식', '가격', '상승', '주택', '펀드', '거래', '규제', '채권', '은

위에서 학인한 토픽의 주요 키워드와, 각 문서에서 지배적으로 사용된 토픽을 확인하여 분류하여 토픽을 다시 한번 확인했습니다.

결과로 나온 토픽들 중에서 코로나로 생긴 영향에 관한 토픽만 선정하여 사용합니다.

토픽중 대부분 감염병, 백신, 확진자와 관련된 토픽이 많았고, 영향에 관한 토픽은 많지 않았습니다. 

사용할 토픽은 아래와 같습니다.

**15번 토픽**

공연 예술 및 해외 활동 감소에 관한 내용.

**23번 토픽**

초/중/고교 등교 연기에 관한 내용.

**24번 토픽**

사회적 거리 두기에 따른 시설 폐쇄 및 휴관에 관한 내용.

**25번 토픽**

긴급 재난 지원금 지급에 관련된 내용.

**28번 토픽**

졸업식 연기 및 행사 취소 및 소비 촉진 정책 연기와 관련된 내용.

**61번 토픽**

온라인 쇼핑 매출 증가에 관련된 내용

**62번 토픽**

온라인 수업 온라인 강의 등록금 인하 관련 내용.

**64번 토픽**

항공 노선 감축 및 운영 중단 관련 내용.

**65번 토픽**

대출, 기업 / 소상공인 지원, 임대료 관련 내용.

**67번 토픽**

기업의 부품 공급 차질, 공장 중단 관련 내용.

---

### 워드 클라우드

각 토픽들을 워드 클라우드를 생성하여 주제를 확인해 보았습니다. 워드 클라우드는 텍스트 마이닝에서 가장 대표적인 시각화 방법입니다. 워드 클라우드를 활용하여 토픽의 전체적인 느낌을 확인할 수 있습니다. 

워드클라우드는 출현빈도만을 보기 때문에 문맥이 고려되는 다른 분석 방법 보다는 제약이 많습니다.

전체 기사를 토큰화 한 데이터는 있지만, 여기서는 토픽별로 분류하여 토큰화된 데이터가 필요하기 때문에, 토큰화를 진행합니다. 과정은 이전과 같습니다.

In [ ]:
# 토큰화를 위한 KoNLPy, Mecab 설치
!set -x \
&& pip install -q konlpy \
&& curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh | bash -x > /dev/null

# 글꼴 설치
!apt-get install -qy fonts-nanum*
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf
font_path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'


In [ ]:
import numpy as np
import pandas as pd
import re, os
import pickle
import matplotlib.pyplot as plt
from konlpy.tag import Mecab
from PIL import Image
from collections import Counter
from wordcloud import WordCloud
import matplotlib as mpl
import matplotlib.font_manager as fm

# 폰트 설정
font_path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'  
font_name = fm.FontProperties(fname=font_path, size=10).get_name()
plt.rc('font', family=font_name)
mpl.rc('font', family=font_name)

In [ ]:
# 폴더에 있는 파일 목록을 가져오는 메서드
def get_filelist(path, extension):
  flist = os.listdir(path)
  result = []
  if path.endswith('/') == False:
    path += '/'
  for file in flist:
    if file.endswith(f'.{extension}'):
      result.append(path+file)
  return sorted(result)

# 2차원 배열 -> 1차원 배열
def flatten(l):
    flatList = []
    for elam in l:
        if type(elam) == list:
            for e in elam:
                flatList.append(e)
        else:
            flatList.append(elam)
    return flatList


In [ ]:
# 사용할 토픽 번호
# topic_use = [15, 23, 24, 25, 28, 61, 62, 64, 65, 67]
topic_use = [14, 22, 23, 24, 27, 60, 61, 63, 64, 66]

In [ ]:
# 토픽 설명 - 범례 및 툴팁에서 사용
description = [
  '15. 공연 예술 및 해외 활동 감소에 관한 내용.',
  '23. 초/중/고교 등교 연기에 관한 내용.',
  '24. 사회적 거리 두기에 따른 시설 폐쇄 및 휴관에 관한 내용.',
  '25. 긴급 재난 지원금 지급에 관련된 내용.',
  '28. 졸업식 연기 및 행사 취소 및 소비 촉진 정책 연기와 관련된 내용.',
  '61. 온라인 쇼핑 매출 증가에 관련된 내용.',
  '62. 온라인 수업, 온라인 강의, 등록금 인하 관련 내용.',
  '64. 항공 노선 감축 및 운영 중단 관련 내용.',
  '65. 대출, 기업 / 소상공인 지원, 임대료 관련 내용.',
  '67. 기업의 부품 공급 차질, 공장 중단 관련 내용.'
]

In [ ]:
# 특수문자를 제거하고 단어만 사용하기 위한 정규표현식
r = re.compile(r'[^ ㄱ-ㅣ가-힣|0-9|a-zA-Z]+', flags=re.U | re.M)

mecab = Mecab()

# 공백 제거 
def clean_text(text):
    text = text.replace(".", " ").strip()
    text = text.replace("·", " ").strip()
    text = r.sub('', string=text)
    return text

# NNP, NNG, SL만 분리
def get_nouns(sentence):
  return [morphs[0] for morphs in mecab.pos(sentence) if morphs[1] in ['NNP', 'NNG',  'SL'] and len(morphs[0]) > 1]

# tokenizer 실행 함수
def tokenize(df):
    processed_data = []
    # for sent in tqdm(df['text']):
    for sent in df['text']:
        sentence = clean_text(str(sent).replace('\n', '').strip())
        processed_data.append(get_nouns(sentence))
    return processed_data

In [ ]:
# lda로 분류된 결과를 토큰화 하여 아래 폴더에 저장
os.makedirs(root + 'lda72_tokenized_all', exist_ok=True)
os.chdir(root + 'lda72_tokenized_all')

# 토픽별로 분류했던 뉴스기사 목록을 각각 토큰화 수행.
# for topic in topic_use:
for topic in range(72):
  num = '{:02}'.format(topic)
  rows = pd.read_csv(f'{root}lda72/{num}.csv')
  with open(f'{topic}.pkl', 'wb') as f:
    pickle.dump(tokenize(rows), f)

'코로나', '확진', '바이러스'와 같은 단어들은 어느 토픽에서는 높은 빈도로 출현하기 때문에 토픽의 특징이 살아나지 못해 제외합니다.

토큰화된 전체 목록에서 빈도수로 상위 50개의 단어를 불용어로 지정했습니다.

불용어로 지정한 단어 목록:

In [ ]:
# 전체 기사 토큰화 목록 불러오기
flist = get_filelist(root + 'tokenized', 'pkl')

In [ ]:
# Counter를 이용하여 전체 언급 빈도 계산
counter_all = Counter()

for file in flist:
  pkl = pickle.load(open(file, 'rb'))
  counter_all.update(flatten(pkl))

In [ ]:
# 전체 단어들 중 언급 순위 50개 단어 사용
stopwords = counter_all.most_common(n=50)
stopwords = [t[0] for t in stopwords]

In [ ]:
print(stopwords)

['코로나', '확진', '바이러스', '신종', '지역', '환자', '감염', '중국', '감염증', '방역', '확산', '지원', '정부', '검사', '서울', '발생', '상황', '미국', '관련', '병원', '마스크', '판정', '이날', '격리', '사회', '시장', '경제', '조사', '추가', '확인', '기업', '이후', '가능', '한국', '경우', '조치', '진행', '대구', '시설', '사람', '교회', '국내', '센터', '이번', '오후', '당국', '사태', '대책', '세계', '예정']


위에서 언급했던 토픽들을 사용하여 워드클라우드를 생성했습니다.

`WordCloud` 패키지를 사용하였습니다.

In [ ]:
# 폰트 옵션
mpl.rcParams['axes.unicode_minus'] = False

fontprop = fm.FontProperties(fname=font_path, size=18)

In [ ]:
# subplot을 생성하기 위한 준비
fig, axs = plt.subplots(5, 2, figsize=(20,50), facecolor='w', edgecolor='k')
fig.subplots_adjust(hspace = .05, wspace = .01)
axs = axs.ravel()

for i, topic in enumerate(topic_use):
  # 각 토픽을 불러와 Counter로 출현 빈도를 셈
  pkl = pickle.load(open(f'{root}lda72_tokenized_all/{topic}.pkl', 'rb'))
  counter = Counter(flatten(pkl))

  # 불용어 제거
  for f in stopwords:
    try:
      counter.pop(f)
    except: 
      continue
  
  # 워드 클라우드 생성
  wc = WordCloud(font_path=font_path, width=1000, height=1000, background_color="white",
                 prefer_horizontal=1.0, max_font_size=300)
  wc.generate_from_frequencies(counter)

  # recolor = wc.recolor(color_func=recolor_func, random_state=True)

  # subplot에 그리기
  axs[i].axis("off")
  axs[i].set_title(description[i], fontproperties=fontprop)
  axs[i].imshow(wc, interpolation="bilinear")

  # 파일 저장
  # fname = file.split('/')[-1]
  # plt.savefig(f'{fname}.png', dpi=300)

Output hidden; open in https://colab.research.google.com to view.

불용어를 처리하기 이전에는 대부분 '코로나', '바이러스' 로 모든 토픽이 가득차는 모습이였으나, 어느정도 불용어를 제거하고 워드 클라우드를 생성하고 보니 단어가 적절히 표현되어 각 토픽의 주제를 확인할 수 있었습니다.

---

### Pie Chart

각 토픽들의 비중을 확인해 보았습니다.

In [ ]:
from math import pi
import pandas as pd

from bokeh.io import output_file, show, output_notebook
from bokeh.palettes import Category20c
from bokeh.models import Legend, HoverTool, Wedge
from bokeh.plotting import figure
from bokeh.transform import cumsum

In [ ]:
# 토픽 설명 - 범례 및 툴팁에서 사용
description = [
  '15. 공연 예술 및 해외 활동 감소에 관한 내용.',
  '23. 초/중/고교 등교 연기에 관한 내용.',
  '24. 사회적 거리 두기에 따른 시설 폐쇄 및 휴관에 관한 내용.',
  '25. 긴급 재난 지원금 지급에 관련된 내용.',
  '28. 졸업식 연기 및 행사 취소 및 소비 촉진 정책 연기와 관련된 내용.',
  '61. 온라인 쇼핑 매출 증가에 관련된 내용.',
  '62. 온라인 수업 온라인 강의 등록금 인하 관련 내용.',
  '64. 항공 노선 감축 및 운영 중단 관련 내용.',
  '65. 대출, 기업 / 소상공인 지원, 임대료 관련 내용.',
  '67. 기업의 부품 공급 차질, 공장 중단 관련 내용.'
]

기존과 동일한 dataframe에서 토픽을 구분지을 topic column을 추가했습니다.

In [ ]:
# 사용할 토픽 선언
topic_use = [14, 22, 23, 24, 27, 60, 61, 63, 64, 66]

# 사용할 토픽들로 이루어진 dataframe 생성
# 각 토픽을 불러와 topic column을 추가하고 하나의 dataframe으로 합침
topic_df = pd.DataFrame()
for num in topic_use:
  tmp_df = pd.read_csv(root+'lda72/%02d.csv' % num)
  c = tmp_df.columns.values
  c[0] = 'Document_No'
  tmp_df.columns = c
  tmp_df = tmp_df.set_index('Document_No')
  tmp_df['topic'] = num
  topic_df = pd.concat([topic_df, tmp_df])

In [ ]:
topic_df.head()

,title,date,category,text,press,link,topic
Document_No,,,,,,,
12,"'미스트롯' 의정부 공연 취소 ""무대 구조물 무너져"" 사과",2020-01-05 16:00:51,생활,'내일은 미스트롯' 전국투어 시즌2 청춘 공연 포스터 2020.01.05. ...,3,https://m.news.naver.com/read.nhn?mode=LSD&mid...,14
13,"미스트롯 전국투어 의정부 공연, 안전 문제로 취소",2020-01-05 16:46:19,생활,"\t\t\t공연기획사 ""무대 설치 도중 구조물 일부 무너져""미스트롯 전국투어 콘서트...",1,https://m.news.naver.com/read.nhn?mode=LSD&mid...,14
21,"익산시, 중앙동 도시재생·문화예술공간 조성 협약",2020-01-06 16:14:21,사회,=전북 익산시는 중앙동 문화예술공간 조성을 통한 도시재생 뉴딜사업의 성공 추진을...,3,https://m.news.naver.com/read.nhn?mode=LSD&mid...,14
22,"익산시, SM엔터 등과 '아이돌 양성소' 만든다…협약 체결",2020-01-06 16:34:17,사회,"옛 하노바호텔에 아카데미, 댄스 연습실 등 조성6일 전북 익산시청 상황실에서 중앙동...",421,https://m.news.naver.com/read.nhn?mode=LSD&mid...,14
271,"영종 청소년가요제·월드뮤직경연대회, 2020년 첫 예선 성황리 개최",2020-01-12 20:05:00,경제,\t\t\t 2020 제1회 영종 월드뮤직경연대회 예선광경/사진=공연&문화허브 M터...,8,https://m.news.naver.com/read.nhn?mode=LSD&mid...,14


토픽을 기준으로 묶어 개수를 셉니다.

In [ ]:
# 토픽을 기준으로 묶어 개수를 셈
t = topic_df.groupby('topic').count()

In [ ]:
# 출현 빈도를 dataframe으로 표현
value = pd.DataFrame(t.T.loc['title'])

In [ ]:
value.head()

,title
topic,
14,7784
22,18706
23,9930
24,15601
27,6227


`Bokeh`를 활용하여 그래프를 그렸습니다.

In [ ]:
output_notebook()



# angle: 파이를 그릴때 사용될 넓이값
# color: 각 토픽의 컬러 지정
# percentage: 각 토픽의 비율
# description: 범례

value['angle'] = value['title'] / value['title'].sum() * 2 * pi
value['color'] = Category20c[len(value)]
value['percentage'] = value['title'] / value['title'].sum()
value['description'] = description

# plot 선언, 크기 지정
p = figure(title="Pie Chart", toolbar_location=None, x_range=(-0.5, 2.0),  sizing_mode='stretch_width')

# 툴팁 설정
hover = HoverTool(
    tooltips = [
           ('topic', '@description'),
          ('value', '@percentage{%F}'),
    ])
p.add_tools(hover)

# 차트 추가
p1 = p.wedge(x=0, y=1, radius = 0.4, start_angle=cumsum('angle', include_zero=True),
             end_angle=cumsum('angle'), legend_field='description', line_color='white', fill_color='color', hover_alpha = 1.0, fill_alpha = 0.9, source=value)


In [ ]:
# 축 숨김, grid 숨김
p.axis.axis_label = None
p.axis.visible = False
p.grid.grid_line_color = None
p.legend.label_text_font_size = '8pt'
show(p)

돈과 관련된 주제(긴급 재난 지원금, 대출, 기업 지원 등)가 가장 비중이 높을 것이라고 생각했지만 교육과 관련된 주제(등교, 온라인 수업, 졸업 연기 등)도 많이 보였습니다. 

사회적 거리두기 또한 많은 이슈를 불러모았을 것이라고 생각했으나 관심은 비교적 적었던 것 같습니다.



---

### 토픽별 시계열 분석

앞서, 선정했던 관련성 있는 토픽들의 언급을 시계열 분석해 보았습니다.

In [ ]:
!pip install -q bokeh
from bokeh.plotting import output_notebook, figure, show
from bokeh.palettes import viridis, Category20c
from bokeh.models import ColumnDataSource, DatetimeTickFormatter, Legend, LegendItem, HoverTool
from gensim.models.ldamodel import LdaModel
from gensim.corpora import Dictionary

import datetime

In [ ]:
# 사용할 토픽 선언
topic_use = [14, 22, 23, 24, 27, 60, 61, 63, 64, 66]

In [ ]:
# 토픽 설명 - 범례 및 툴팁에서 사용
description = [
  '15. 공연 예술 및 해외 활동 감소에 관한 내용.',
  '23. 초/중/고교 등교 연기에 관한 내용.',
  '24. 사회적 거리 두기에 따른 시설 폐쇄 및 휴관에 관한 내용.',
  '25. 긴급 재난 지원금 지급에 관련된 내용.',
  '28. 졸업식 연기 및 행사 취소 및 소비 촉진 정책 연기와 관련된 내용.',
  '61. 온라인 쇼핑 매출 증가에 관련된 내용.',
  '62. 온라인 수업 온라인 강의 등록금 인하 관련 내용.',
  '64. 항공 노선 감축 및 운영 중단 관련 내용.',
  '65. 대출, 기업 / 소상공인 지원, 임대료 관련 내용.',
  '67. 기업의 부품 공급 차질, 공장 중단 관련 내용.'
]

사용되는 토픽은 다음과 같습니다.

In [ ]:
description

['15. 공연 예술 및 해외 활동 감소에 관한 내용.',
 '23. 초/중/고교 등교 연기에 관한 내용.',
 '24. 사회적 거리 두기에 따른 시설 폐쇄 및 휴관에 관한 내용.',
 '25. 긴급 재난 지원금 지급에 관련된 내용.',
 '28. 졸업식 연기 및 행사 취소 및 소비 촉진 정책 연기와 관련된 내용.',
 '61. 온라인 쇼핑 매출 증가에 관련된 내용.',
 '62. 온라인 수업 온라인 강의 등록금 인하 관련 내용.',
 '64. 항공 노선 감축 및 운영 중단 관련 내용.',
 '65. 대출, 기업 / 소상공인 지원, 임대료 관련 내용.',
 '67. 기업의 부품 공급 차질, 공장 중단 관련 내용.']

사용할 Dataframe은 이전과 동일합니다.

In [ ]:
topic_df = pd.DataFrame()
# 데이터 프레임에 토픽 column을 추가하여 토픽을 합침 - 이전과 동일
for num in topic_use:
  tmp_df = pd.read_csv(root+'lda72/%02d.csv' % num)
  c = tmp_df.columns.values
  c[0] = 'Document_No'
  tmp_df.columns = c
  tmp_df = tmp_df.set_index('Document_No')
  tmp_df['topic'] = num
  topic_df = pd.concat([topic_df, tmp_df])

In [ ]:
topic_df.head()

,title,date,category,text,press,link,topic
Document_No,,,,,,,
12,"'미스트롯' 의정부 공연 취소 ""무대 구조물 무너져"" 사과",2020-01-05 16:00:51,생활,'내일은 미스트롯' 전국투어 시즌2 청춘 공연 포스터 2020.01.05. ...,3,https://m.news.naver.com/read.nhn?mode=LSD&mid...,14
13,"미스트롯 전국투어 의정부 공연, 안전 문제로 취소",2020-01-05 16:46:19,생활,"\t\t\t공연기획사 ""무대 설치 도중 구조물 일부 무너져""미스트롯 전국투어 콘서트...",1,https://m.news.naver.com/read.nhn?mode=LSD&mid...,14
21,"익산시, 중앙동 도시재생·문화예술공간 조성 협약",2020-01-06 16:14:21,사회,=전북 익산시는 중앙동 문화예술공간 조성을 통한 도시재생 뉴딜사업의 성공 추진을...,3,https://m.news.naver.com/read.nhn?mode=LSD&mid...,14
22,"익산시, SM엔터 등과 '아이돌 양성소' 만든다…협약 체결",2020-01-06 16:34:17,사회,"옛 하노바호텔에 아카데미, 댄스 연습실 등 조성6일 전북 익산시청 상황실에서 중앙동...",421,https://m.news.naver.com/read.nhn?mode=LSD&mid...,14
271,"영종 청소년가요제·월드뮤직경연대회, 2020년 첫 예선 성황리 개최",2020-01-12 20:05:00,경제,\t\t\t 2020 제1회 영종 월드뮤직경연대회 예선광경/사진=공연&문화허브 M터...,8,https://m.news.naver.com/read.nhn?mode=LSD&mid...,14


이것을 date / topic을 기준으로 그룹화하여 pivot을 생성합니다.

여기에서 토픽 번호가 다른 것은 gensim의 ldamodel 에서는 토픽 번호가 0번부터 pyLDAvis 에서는 1번부터 시작하기 때문입니다.

In [ ]:
# 'date' column을 날짜 타입으로 변경
topic_df['date'] = pd.to_datetime(topic_df['date'])

In [ ]:
# 기간, 토픽을 기준으로 그룹
t = topic_df.groupby([pd.Grouper(key='date', freq='W'), 'topic']).count()

In [ ]:
# 두 기준으로 그룹화 하였던 것으로 pivot으로 생성
t = t.reset_index().pivot(index='date', columns='topic', values='title').fillna(0)
t = t[ t.index < '2020-09-01' ]

In [ ]:
t.tail()

topic,14,22,23,24,27,60,61,63,64,66
date,,,,,,,,,,
2020-08-02,208.0,471.0,236.0,295.0,12.0,489.0,389.0,163.0,266.0,103.0
2020-08-09,221.0,140.0,139.0,155.0,11.0,415.0,449.0,140.0,293.0,204.0
2020-08-16,234.0,305.0,258.0,240.0,51.0,436.0,342.0,76.0,256.0,95.0
2020-08-23,373.0,636.0,927.0,255.0,182.0,437.0,531.0,98.0,158.0,108.0
2020-08-30,245.0,968.0,730.0,340.0,135.0,649.0,739.0,93.0,294.0,154.0


시각화는 `Bokeh`를 활용하여 라인 차트를 그렸습니다. 각 토픽의 시기별 출현 빈도를 확인할 수 있습니다.

In [ ]:
output_notebook()

# plot 생성, 드래그 방지
plot = figure(title='topic', plot_width=1000, plot_height=400, x_axis_type='datetime', sizing_mode='stretch_width')
plot.toolbar.active_drag = None

In [ ]:
# 색상 지정
colors = Category20c[len(t.columns)]

# 범례 저장 list
legend_list = []

# 토픽별로 lineplot을 그림
for i, (item, color) in enumerate(zip(t.iteritems(), colors)):
  c = plot.line(x = item[1].index, y = item[1].values, color=color, line_alpha = 0.8, hover_line_width= 4, hover_alpha = 1.0, line_width = 2)
  legend_list.append([ description[i], [c] ])
  
  # Plotdp 툴팁 추가.
  plot.add_tools(HoverTool(
      renderers = [c], 
      tooltips = [
                  ('date', '$x{%F}'),
                  ('topic', description[i]),
                  ('value', '$y{D}'),
                  ],
      formatters={
        '$x': 'datetime'
    }))

In [ ]:
# 범례 생성
legend = Legend(items=legend_list, level = 'annotation')
legend.click_policy = 'hide'


# plot 오른쪽에 범례 추가
plot.add_layout(legend, 'right')


# 범례 크기 조절
plot.legend.label_text_font_size = '11px'

In [ ]:
show(plot)

졸업식 연기 관련 토픽은 졸업식 / 입학식 부근인 2/3/8월에 수치가 높았으나, 이외에는 적은 모습을 보여줬습니다. 예상보다는 많은 수가 집계 되었는데, 분류가 완벽하지 않아 생긴 모습으로 판단됩니다.

64, 65, 67번 토픽은 각각 다른 주제의 내용이고, 절대적인 기사의 건수는 다르지만 그래프는 대체로 비슷한 모습을 보여줬습니다. 

온라인 수업 관련 토픽은 5월 30일 여느때 다른 토픽들보다 높은 총량을 보여주었는데, 이때, 집담 감염 문제가 있었고, 이에 따라 등교 지침이 내려졌기 때문에 급증했던 것으로 판단됩니다.

---

## 2.5 Word to Vector를 활용한 연관어 시각화

워드 투 벡터를 이용하여 연관어 분석을 수행했습니다.

`gensim`의 `word2vec`을 이용하여 단어를 벡터화 하였고, 이를 PCA를 이용하여 2차원으로 축소한 후 시각화 하였습니다.

In [ ]:
from gensim.models.word2vec import Word2Vec
import pickle
from sklearn.decomposition import PCA
from collections import Counter
import os

먼저, word2vec 모델을 생성합니다.

토픽을 생성할때에 토큰화된 단어 목록으로 학습을 수행했습니다.

또한, 이전에 선별되었던 토픽의 기사만을 이용하여 범위를 축소하였습니다.

모델 생성은 `gensim`에서 제공되는 `Word2Vec` 클래스를 이용합니다. 

CBOW가 아닌 skip-gram을 사용했습니다.

In [ ]:
# 폴더에 있는 파일 목록을 가져오는 메서드
def get_filelist(path, extension):
  flist = os.listdir(path)
  result = []
  if path.endswith('/') == False:
    path += '/'
  for file in flist:
    if file.endswith(f'.{extension}'):
      result.append(path+file)
  return sorted(result)

# 여러 파일의 토큰을 제너레이터로 생성
class TokenCorpus():
  def __init__(self, flist):
    self.flist = flist

  def __iter__(self):
    for file in self.flist:
      with open(file, 'rb') as f:
        for el in pickle.load(f):
          yield el
          
# 2차원 배열 -> 1차원 배열
def flatten(l):
    flatList = []
    for elam in l:
        if type(elam) == list:
            for e in elam:
                flatList.append(e)
        else:
            flatList.append(elam)
    return flatList


In [ ]:
# 사용할 토픽 선언
topic_use = [14, 22, 23, 24, 27, 60, 61, 63, 64, 66]

corpus = TokenCorpus(flist)

In [ ]:
# 전체 토큰화된 것들 중에서 선별한 토픽만을 이용함.
flist = get_filelist(root + 'lda72_tokenized_all', 'pkl')
flist = [ flist[topic]  for topic in topic_use]

In [ ]:
# Word2Vec 모델 생성
# 200차원의 벡터로 skip-gram으로 학습
model = Word2Vec(corpus, size=200, window=5, min_count=2000, sg=1)
model.init_sims(replace=True)

model.save(root+'w2v.model')

In [ ]:
print(f'vector size: {model.vector_size}')

vector size: 200


In [ ]:
# 모든 단어의 벡터 값을 가져옴
word_vectors = model.wv
vocabs = word_vectors.vocab.keys()
word_vectors_list = [word_vectors[v] for v in vocabs]

In [ ]:
!pip install bokeh

In [ ]:
from bokeh.plotting import Figure, output_notebook, show
from bokeh.models import  ColumnDataSource, Select, CustomJS
from bokeh.palettes import viridis
from bokeh.layouts import layout, column

from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np

output_notebook()

2차원으로 표현하기 위해 200차원의 워드 벡터를 `PCA` 알고리즘을 이용하여 2차원으로 차원을 축소하였습니다. 

In [ ]:
# PCA를 이용하여 2차원으로 축소
pca = PCA(n_components = 2)
xys = pca.fit_transform(word_vectors_list)

In [ ]:
# plot에서 x축, y축
xs = xys[:, 0]
ys = xys[:, 1]

In [ ]:
print(f'x: {xs[:5]}')
print(f'y: {ys[:5]}')

x: [-0.14210817 -0.09550632 -0.01150816  0.23729176 -0.14441065]
y: [-0.04496436 -0.10240583 -0.2011007  -0.01415945 -0.08759753]


In [ ]:
# 사용할 데이터들을 dataframe으로 묶음
df = pd.DataFrame()
df['x'] = xs
df['y'] = ys
df['annotate'] = list(vocabs)
df['color'] = 'grey'
df['alpha'] = 0.2

출현 빈도별로 크기를 다르게 하기 위해 Counter를 이용하여 단어의 빈도를 계산하고, 이를 `MinMaxScaler` 활용해 정규화합니다.

In [ ]:
# Counter를 이용하여 전체 언급 빈도 계산
counter = Counter()

for file in flist:
  pkl = pickle.load(open(file, 'rb'))
  counter.update(flatten(pkl))

In [ ]:
counter.most_common(5)

[('코로나', 371780),
 ('확진', 188296),
 ('바이러스', 107398),
 ('지역', 93667),
 ('신종', 92268)]

In [ ]:
d = pd.DataFrame(index=counter.keys(), data = counter.values(), columns = ['value'])
d = d.sort_index()
df = df.sort_values(by='annotate')

In [ ]:
d = d[d.index.isin(df['annotate'])]

minmax = MinMaxScaler(feature_range=(5, 25))
value = minmax.fit_transform(d)
value = value.round(3)

In [ ]:
df['size'] = value

그래프를 그릴때 사용할 데이터를 dataframe으로 구조화하였습니다.

In [ ]:
df.head()

,x,y,annotate,color,alpha,size
713,0.055800,0.340743,AFP,grey,0.2,6.058
609,0.019441,0.376587,AP,grey,0.2,6.046
1227,-0.004678,0.462070,CNN,grey,0.2,6.042
1439,0.213932,0.216895,EU,grey,0.2,6.012
253,-0.095378,0.028138,KBS,grey,0.2,6.075


`Bokeh`를 이용해 시각화를 수행하였습니다. 
각 단어를 선택하면 해당 단어와 유사한 단어들이 보라색으로 표시됩니다.

bokeh는 plot들이 JS로 렌더링됩니다. 따라서 파이썬 객체 word2vec 모델을 그대로 사용할 수 없기 때문에 `word2vec.wv.most_similar` 결과를 모두 딕셔너리에 담아 사용합니다.

word2vec 모델의 관점에서 해당 단어와 유사한 단어라는 뜻인데, 이는 코사인 유사도를 계산하여 유사도가 가장 높은 값을 사용합니다.

개수는 20개로 고정입니다.

선택 메뉴의 단어들은 각 토픽의 대표적인 단어를 10개씩 뽑아 사용했습니다.

In [ ]:
# 시각화에서 사용할 각 단어의 most_similar 값을 저장
word_similar = dict()
for word in vocabs:
   word_similar[word] = [w[0] for w in model.wv.most_similar(word, topn=20)]

In [ ]:
keyword_list = list(set(['공연', '예술', '영화', '문화', '코로나', '관객', '극장', '음악', '무대', '콘서트', '학교', '학생', '등교', '수업', '개학', '교육청', '학년', '교육', '교사', '교육부',  '거리', '두기', '사회', '시설', '운영', '생활', '코로나', '단계', '이용', '도서관',  '지급', '지원', '지원금', '고용', '재난', '긴급', '신청', '소득', '코로나', '가구',  '코로나', '확산', '취소', '신종', '연기', '바이러스', '단계', '감염증', '결정', '예정','판매', '매출', '온라인', '제품', '고객', '상품', '구매', '매장', '소비자', '코로나', '대학', '시험', '학생', '교육', '온라인', '수업', '코로나', '채용', '강의', '진행',  '항공', '운항', '노선', '항공사', '코로나', '중단', '아시아나항공', '국제선', '공항',   '지원', '대출', '코로나', '기업', '자금', '은행', '금융', '공인', '소상', '중소기업', '공장', '생산', '자동차', '울산', '업체', '가동', '코로나', '현대', 'LG', '현대차']))

In [ ]:
# datatype을 bokeh에 맞게 변환
source = ColumnDataSource(data=dict(
    x = df['x'],
    y = df['y'],
    color = df['color'],
    annotate = df['annotate'],
    alpha = df['alpha'],
    size = df['size']
))

In [ ]:
# 표시되는 원에 글자 툴팁
tooltips = [
            ('word', '@annotate')
]

# 그래프 그리기. 축 제거
p = Figure(title="sample", sizing_mode='stretch_width', tooltips = tooltips)
p.axis.visible = False

In [ ]:
# 메뉴 선택시에 변경될 데이터 JS 콜백으로 구현
callback = CustomJS(args=dict(source=source, word_similar=word_similar),
                    code="""
  var data = source.data;
  var word_similiar = word_similiar;
  var value = cb_obj.value;
  var color = data['color'];
  var annotate = data['annotate'];
  var alpha = data['alpha'];
  var size = data['size'];
  for (var i = 0; i < size.length; i++) {
    if (word_similar[value].includes(annotate[i])) {
      color[i] = 'purple';
      alpha[i] = 0.6;

    } else {
      color[i] = 'grey';
      alpha[i] = 0.2;

    }
  }
  console.log(source);
  source.change.emit();
  """ )

In [ ]:
# 단어를 선택할 Select
menu = Select(title="keyword", options=sorted(keyword_list), value="")
menu.js_on_change('value', callback)

In [ ]:
# 위 source를 기반으로 그래프를 그림
p.circle(x='x', y='y', size = 'size', color='color', fill_alpha= 'alpha', line_width = 0, source=source)
layout = column(menu, p)
layout.sizing_mode = 'stretch_width'

In [ ]:
show(layout)

word2vec를 통해서 연관어를 확인해 보았습니다. 

선정된 키워드와 유사한 키워드를 강조하여 2차원으로 표시하였으나, 시각화 결과가 주제와 밀접하게 관련되었다고 보기는 어려웠습니다

하지만, 기사에서 많이 언급되는 키워드가 벡터에서도, 위치에도 반영이 되기 때문에 계속해서 분석 또는 학습을 진행할 때에 도움이 될 것으로 보입니다.

---

# 3. 결론

- 뉴스기사 건수 및 본문을 바탕으로 시기별로 코로나 관련하여 어떤 관심사가 높아졌는지 알 수 있었습니다.

- 토픽 모델링을 통해 다양한 코로나 관련 기사들을 볼 수 있었고, 이를 분류하여 원하는 토픽을 확인하였습니다.

- 사람들의 어떤 것들에 관심을 가지고, 또 관심이 어떻게 이동하는지 확인할 수 있었습니다.